# Saving Beta Bank: Predicting Customer Churn with Machine Learning
## Project Overview & Objective
Recently, Beta Bank has been gradually losing customers, which has a negative financial impact on the bank.  Given the costs associated with new account creation, it is much more economically viable to maintain existing customers than try to acquire new ones.

This project will create a machine learning model that predicts the likelihood that a customer will leave the bank soon based on data for current and former clients.  The model will be considered reliable upon achieving an F1 score of 0.59 or better, as this score indicates that the model is predicting classes without a high number of false negatives or false positives.  Beta Bank can use this model to effectively predict which customers are likely to leave to deploy targeted client retention strategies aimed at those customers.

## Data Description & Preparation

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

In [4]:
customers = pd.read_csv('/datasets/Churn.csv')
customers

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3.0,75075.31,2,1,0,92888.52,1


In [5]:
customers.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,9091.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,4.997690,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.894723,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [6]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


The data set contains data on 10,000 customers of Beta Bank.  Specifically, the data contains their unique customer id, surname, credit score, country (geography), gender, age, period of maturation for their account's fixed deposit (tenure), account balance, number of products, whether or not they have a credit card from the bank, whether or not they are considered an active member, their estimated salary, and whether or not they've left the bank. All data is intact with the exception of some missing data in the tenure field, which I will need to investigate further.  Data is primarily numerical, though the data for surnames, geography, and gender are object data type.  Geography and Gender I will convert to numerical via One-Hot Encoding so that these data are factored by the algorithm.  Customer ID, Surname, Row Number do not have an impact on whether or not a customer leaves the bank, so I will drop the columns as the information they contain is not needed for building a model.

Customers of the bank have credit scores that fall across the board, but the average is a fair score of 650.  Customers range from 18 to 92, though the average customer is in their late 30s.  Accounts have various maturation periods from 0 to 10 years with an mean period of 5 years.  Account balances range from 0 to over 250k dollars with the average account having around 76k dollars.  However, given 20 percent of accounts have been cancelled, that number skews a bit low.  The median of 97k dollars is more representative.  This is not too surprising given the average customer makes an estimated 100k dollars per year, though one unfortunate customer reports an annual income of 11 dollars.  Customers have anywhere from 1 to 4 products, though most customers have 1-2.  70 percent of customers have credit cards, but only 52 percent of customers are active users... yikes!  Of all the customers in the data set 20 percent have already left.  Since this is a pressing issue for the bank's future, let's move on.

In [7]:
#changing column headers to lowercase to simplify further analysis
customers.columns = map(str.lower, customers.columns)

In [8]:
customers[customers['tenure'].isnull()]

,rownumber,customerid,surname,creditscore,geography,gender,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,exited
30,31,15589475,Azikiwe,591,Spain,Female,39,NaN,0.00,3,1,0,140469.38,1
48,49,15766205,Yin,550,Germany,Male,38,NaN,103391.38,1,0,1,90878.13,0
51,52,15768193,Trevisani,585,Germany,Male,36,NaN,146050.97,2,0,0,86424.57,0
53,54,15702298,Parkhill,655,Germany,Male,41,NaN,125561.97,1,0,0,164040.94,1
60,61,15651280,Hunter,742,Germany,Male,35,NaN,136857.00,1,0,0,84509.57,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9944,9945,15703923,Cameron,744,Germany,Male,41,NaN,190409.34,2,1,1,138361.48,0
9956,9957,15707861,Nucci,520,France,Female,46,NaN,85216.61,1,1,0,117369.52,1
9964,9965,15642785,Douglas,479,France,Male,34,NaN,117593.48,2,0,0,113308.29,0
9985,9986,15586914,Nepean,659,France,Male,36,NaN,123841.49,2,1,0,96833.00,0


In [9]:
customers[customers['tenure'].isnull()].describe()

,rownumber,customerid,creditscore,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,exited
count,909.000000,9.090000e+02,909.000000,909.000000,0.0,909.000000,909.000000,909.000000,909.000000,909.000000,909.000000
mean,4866.386139,1.568981e+07,648.451045,38.647965,NaN,76117.341474,1.530253,0.710671,0.510451,99180.389373,0.201320
std,2909.604343,7.511225e+04,99.079381,9.785438,NaN,63105.690715,0.588452,0.453701,0.500166,56378.063765,0.401207
min,31.000000,1.556581e+07,359.000000,18.000000,NaN,0.000000,1.000000,0.000000,0.000000,106.670000,0.000000
25%,2311.000000,1.562658e+07,580.000000,32.000000,NaN,0.000000,1.000000,0.000000,0.000000,49872.330000,0.000000
50%,4887.000000,1.568687e+07,647.000000,37.000000,NaN,96674.550000,1.000000,1.000000,1.000000,99444.020000,0.000000
75%,7306.000000,1.575680e+07,718.000000,43.000000,NaN,128554.980000,2.000000,1.000000,1.000000,145759.700000,0.000000
max,10000.000000,1.581569e+07,850.000000,92.000000,NaN,206663.750000,4.000000,1.000000,1.000000,199390.450000,1.000000


There is no immediately clear reason why these 909 accounts do not have tenure info.  Given there is no information upon which to base a guess of their account maturation term but the info is likely quite relevant to predicting whether or not a customer remains with the bank, I will drop these rows, even though that means losing 9% of the data set, so they do not impede later predictions.  Fortunately, this should not affect the predictions later on, as the means for most columns in this subset are close or equivalent to the means for the entire dataset.

In [10]:
customers.dropna(subset=['tenure'], inplace=True)
customers.shape

(9091, 14)

In [11]:
#dropping unneccessary columns
customers.drop(['customerid', 'surname', 'rownumber'], axis=1, inplace=True)

Now that unnecessary columns have been removed, I will convert the geography and gender columns to a binary yes/no format.

In [12]:
#beginning OHE for categorical data
customers = pd.get_dummies(customers, drop_first=True)
customers

,creditscore,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,exited,geography_Germany,geography_Spain,gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
9994,800,29,2.0,0.00,2,0,0,167773.55,0,0,0,0
9995,771,39,5.0,0.00,2,1,0,96270.64,0,0,0,1
9996,516,35,10.0,57369.61,1,1,1,101699.77,0,0,0,1
9997,709,36,7.0,0.00,1,0,1,42085.58,1,0,0,0


Now, the data will reflect which country the customer is from in a binary format.  If Germany and Spain are both 0, the customer is from France.

In [13]:
#checking for duplicates
customers.duplicated().sum()

0

The data should now be ready for beginning to build a model.  In the next section, I will build the model and determine the steps needed to balance the classes and create a more accurate model.

## Initial Model
Before assessing which model will work best, I will check class imbalance and split the data set.  Then, I will build a few different models to determine the best model with which to proceed.

In [14]:
customers['exited'].value_counts(normalize=True)

0    0.796062
1    0.203938
Name: exited, dtype: float64

Consistent with my observation above, 80% of the customers in the data set are still bank customers, but 20% have already left.  This is a fairly imbalanced data set, so I will need to account for that as I revise this initial model.

For further analysis, I will split 60% of the data into a training set to train the model, 20% into a validation set to assist in hyperparameter tuning, and the remaining 20% will serve as a test set to evaluate the model's performance.

In [15]:
train, remain = train_test_split(customers, test_size=0.4, random_state=1)
valid, test = train_test_split(remain, test_size=0.5, random_state=1)
print(f'Training Set Size: {train.shape[0]}')
print(f'Validation Set Size: {valid.shape[0]}')
print(f'Test Set Size: {test.shape[0]}')

Training Set Size: 5454
Validation Set Size: 1818
Test Set Size: 1819


In [16]:
#defining variables
target = customers['exited']
features = customers.drop(['exited'], axis=1)
train_target = train['exited']
train_features = train.drop(['exited'], axis=1)
valid_target = valid['exited']
valid_features = valid.drop(['exited'], axis=1)
test_target = test['exited']
test_features = test.drop(['exited'], axis=1)
print(train_features.shape)
print(train_target.shape)
print(valid_features.shape)
print(valid_target.shape)
print(test_features.shape)
print(test_target.shape)

(5454, 11)
(5454,)
(1818, 11)
(1818,)
(1819, 11)
(1819,)


Now that the data set is split and the target and features defined for each subset, I will begin with a decision tree model to get a baseline level of accuracy for the dataset.  Decision trees generally have low accuracy, but high processing speed, so it is a good starting point.

In [17]:
#creating decision tree model
for depth in range(5, 16):
    dt_model = DecisionTreeClassifier(max_depth=depth, random_state=1)
    dt_model.fit(train_features, train_target)
    train_predictions = dt_model.predict(train_features)
    valid_predictions = dt_model.predict(valid_features)
    print('max_depth =', depth, ': ', end='')
    print(f1_score(valid_target, valid_predictions))

max_depth = 5 : 0.4132231404958678
max_depth = 6 : 0.49084249084249076
max_depth = 7 : 0.5064220183486238
max_depth = 8 : 0.4991452991452991
max_depth = 9 : 0.4775641025641026
max_depth = 10 : 0.5055292259083728
max_depth = 11 : 0.4783950617283951
max_depth = 12 : 0.4868035190615836
max_depth = 13 : 0.486090775988287
max_depth = 14 : 0.4774381368267831
max_depth = 15 : 0.47765363128491617


The decision tree model achieves the highest F1 score with a tree depth of 7: 0.51.  The model performs ok!  Still, a Random Forest model or Linear Regression model might perform better.

In [ ]:
# creating random forest model
best_score = 0
best_est = 0
for est in range(160, 260):
    rf_model = RandomForestClassifier(random_state=1, n_estimators=est)
    rf_model.fit(train_features, train_target)
    score = f1_score(valid_target, rf_model.predict(valid_features))
    if score > best_score:
        best_score = score
        best_est = est
print("F1 of the best model on the validation set (n_estimators = {}): {}".format(best_est, best_score))

The Random Forest model is able to improve F1 substantially to 0.56 with 169 estimators!

In [ ]:
# creating logistic regression model

scaler = StandardScaler()
train_features_scaled = scaler.fit_transform(train_features)
valid_features_scaled = scaler.transform(valid_features)

lr_model = LogisticRegression(random_state=1, solver='liblinear')
lr_model.fit(train_features_scaled, train_target)
train_score = f1_score(train_target, lr_model.predict(train_features_scaled))
valid_score = f1_score(valid_target, lr_model.predict(valid_features_scaled))
print("F1 of the logistic regression model on the training set:", train_score)
print("F1 of the logistic regression model on the validation set:", valid_score)

Logistic regression is not much help here, even with feature scaling, so I will proceed with the Random Forest model.  It has a solid F1, though could use some improvements with better class balancing.  Before working to improve the model by taking class imbalance into account, I will calculate some benchmarks in addition to F1.

In [ ]:
model = RandomForestClassifier(random_state=1, n_estimators=169)
model.fit(train_features, train_target)
valid_predicted = pd.Series(model.predict(valid_features))
valid_predicted.value_counts(normalize=True)

The model predicted that 89% of customers would stay with the bank versus 11% leaving.

In [ ]:
valid_proba = model.predict_proba(valid_features)
valid_proba_1 = valid_proba[:, 1]
roc_auc_score(valid_target, valid_proba_1)

The F1 score for the model is ok but can definitely be improved upon by better balancing the classes.  The AUC-ROC score, however, is very good.  The model does well distinguishing between positive and negative classes.

## Improving the Model
While the AUC-ROC score is fairly high, the model's performance can be improved with regard to the F1 value, indicating that class imbalance is affecting precision and recall.  Given that the model predicted 89% of customers would stay and 11% would leave, it would seem the model is overpredicting the number of customers who will stay--not shocking given the data was already imbalanced in that direction.  To attempt to improve the model, I will try several strategies to improve the F1 score to ensure the model is properly sensitive to class imbalance.
### Adding class_weight
Given that this dataset has an 1:4 ratio between clients who have left the bank and clients who are still customers, I will start by adding the class_weight argument to my model to see if it lends any improvement.

In [ ]:
model = RandomForestClassifier(random_state=1, n_estimators=169, class_weight='balanced')
model.fit(train_features, train_target)
valid_predicted = model.predict(valid_features)
f1_score(valid_target, valid_predicted)

In [ ]:
valid_proba = model.predict_proba(valid_features)
valid_proba_1 = valid_proba[:, 1]
roc_auc_score(valid_target, valid_proba_1)

The balanced class weight argument does not improve the model.  In fact, the model performs a bit worse across all metrics.  I will try a different approach to see if I can increase the F1 of the Random Forest model.

### Downsampling
Given the data contains a significantly higher amount of customers who are still with the bank, I will attempt to improve the model by downsampling this population so that the model no longer overestimates the percentage of users who will stay with the bank.

In [ ]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    downsampled_features = pd.concat([features_zeros.sample(frac=fraction, random_state=1)] + [features_ones])
    downsampled_target = pd.concat([target_zeros.sample(frac=fraction, random_state=1)] + [target_ones])
    
    downsampled_features, downsampled_target = shuffle(downsampled_features, downsampled_target, random_state=1)
    
    return downsampled_features, downsampled_target

best_f1 = 0
best_fraction = 0
for fraction in np.arange(0.05, 1.05, 0.05):
    downsampled_features, downsampled_target = downsample(train_features, train_target, fraction)
    
    model = RandomForestClassifier(random_state=1, n_estimators=169)
    model.fit(downsampled_features, downsampled_target)
    valid_predicted = model.predict(valid_features)
    f1_score = f1_score(valid_target, valid_predicted)
    
    if f1_score > best_f1:
        best_f1 = f1_score
        best_fraction = fraction
        
print('Best F1 Score Achieved:', best_f1)
print('Best Fraction:', best_fraction)

In [ ]:
downsampled_features, downsampled_target = downsample(train_features, train_target, 0.5)
model = RandomForestClassifier(random_state=1, n_estimators=169)
model.fit(downsampled_features, downsampled_target)
valid_predicted = model.predict(valid_features)
f1_score(valid_target, valid_predicted)

In [ ]:
valid_proba = model.predict_proba(valid_features)
valid_proba_1 = valid_proba[:, 1]
roc_auc_score(valid_target, valid_proba_1)

Downsampling improved the F1 while maintaining the AUC-ROC.  The model is handling class imbalance better than before!  I will finalize my model adjustments by adjusting the threshold.

### Threshold Adjustment
To ensure the model considers the possibility of a customer leaving the bank more thoroughly, I will lower the threshold to increase recall and align the model more closely with the data set.

In [ ]:
for threshold in np.arange(0, 1.1, 0.1):
    valid_predicted = valid_proba_1 > threshold
    F1 = f1_score(valid_target, valid_predicted)
    AUCROC = roc_auc_score(valid_target, valid_proba_1)
    print('Threshold = {:.2f}, F1 = {:.3f}, AUC-ROC = {:.3f}'.format(threshold, F1, AUCROC))

Threshold adjustment does not provide any improvement to the model; in fact, the argument lowers F1 score.  I will now explore if upsampling can help the model's F1 score.

### Upsampling

In [ ]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    upsampled_features = pd.concat([features_ones] + [features_zeros] * repeat)
    upsampled_target = pd.concat([target_ones] + [target_zeros] * repeat)
    
    upsampled_features, upsampled_target = shuffle(upsampled_features, upsampled_target, random_state=1)
    
    return upsampled_features, upsampled_target

best_f1 = 0
best_repeat = 0
for repeat in range(0, 10):
    upsampled_features, upsampled_target = upsample(train_features, train_target, repeat)
    
    model = RandomForestClassifier(random_state=1, n_estimators=169)
    model.fit(upsampled_features, upsampled_target)
    valid_predicted = model.predict(valid_features)
    f1_score = f1_score(valid_target, valid_predicted)
    
    if f1_score > best_f1:
        best_f1 = f1_score
        best_repeat = repeat
        
print('Best F1 Score Achieved:', best_f1)
print('Best Repeat:', best_repeat)

In [ ]:
valid_proba = model.predict_proba(valid_features)
valid_proba_1 = valid_proba[:, 1]
roc_auc_score(valid_target, valid_proba_1)

Upsampling does not seem to be helping the model, either.  However, downsampling has already helped us achieve the F1 score needed.  Let's see how it performs with the test set.
## Final Test
Now that we have a model that has achieved the desired minimum F1 score, I will run it with our test set to see how it performs.

In [ ]:
model = RandomForestClassifier(random_state=1, n_estimators=169)
model.fit(downsampled_features, downsampled_target)
test_predicted = model.predict(test_features)
f1_score(test_target, test_predicted)

In [ ]:
valid_proba = model.predict_proba(valid_features)
valid_proba_1 = valid_proba[:, 1]
roc_auc_score(valid_target, valid_proba_1)

In [ ]:
test_predicted = pd.Series(test_predicted)
test_predicted.value_counts(normalize=True)

The model performed even better with the test set, achieving an F1 score of 0.61 while maintaining its AUC-ROC score!  Furthermore, its class predictions are much more aligned with the reality currently facing Beta Bank: the model predicts 79% of the dataset will stay with the bank, while 21% will leave... sounds like the customer retention team has at least 1% of customers they need to focus on ASAP!

## Conclusion
Beta Bank's future is in jeopardy as increasing numbers of customers leave the bank.  In an effort to better predict which customers are likely to leave the bank, I developed a machine learning model to predict customer churn so that customer relations can better target their efforts on clients who are likely considering leaving the bank.  The goal was to create a model that achieves an F1 score of 0.59 or better, ensuring accurate predictions with minimal false negatives and false positives.

The final model uses the RandomForestClassifier trained with a downsampled target and downsampled features to address class imbalance and achieve an F1 score of 0.61 on the test set and a very good AUC-ROC score.  Its predictions align closely with the actual distribution of customer churn for the data set.  This tool can now be applied to datasets containing information on current customers to identify on which customers the bank should focus its customer retention strategies.  Hopefully, with the predictions of this model and solid retention strategies, Beta Bank can be saved!